# Computing gradient

Training DNNs requires computing gradient of the loss with respect to the parameters, and this is topic of this notebook.

Your task will be to:

* Implement Evolution Search training of DNNs that does not require backpropagation
* Implement forward and backward functions for Linear and ReQU modules

Goal is to:

* Introduce ingredients necessary to train NN:
    * model
    * loss
    * optimizer
* Introduce different ways of computing or estimating $\frac{\partial{L(w)}}{{\partial w}}$
* Get understanding of Evolution Search and Backpropagation algorithms

Exam:

* For exam you are expected to understand how backpropagation works. Questions might refer to implementation details in PyTorch. See also Resources section for a good video about backpropgation, and how it is implemented in PyTorch.

What's (probably) next:

* Go through each component of the training loop:
    * (Lab 5) Neural Networks practical, part 1: architecture, understanding what is learned
    * (Lab 6, ?) Neural Networks practical part 2: setting up data and loss
    * (Lab 7, ?) Neural Networks practical part 3: optimization

Resources:

* Backprop with focus on PyTorch: https://www.youtube.com/watch?v=ma2KXWblllc (see also other lectures from this series)

* Evolution Search https://eng.uber.com/deep-neuroevolution/, https://arxiv.org/abs/1712.06564

* Matrix derivatives http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/3274/pdf/imm3274.pdf

# SGD requires $\frac{\partial L (w) }{\partial w}$!

<p>
<font size=5>
$$\frac{\partial L (w) }{\partial w} = \frac{1}{K} \sum_{i=1}^{K} \frac{\partial L (x_i, w) }{\partial w}$$
</font>
</p>

<img width=600 src="https://github.com/gmum/nn2018/raw/master/lab/fig/3/sgd.png">

# Setup 

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tqdm
import json

import torch
import torch.nn.functional as F

from torch import optim
from torch import nn
from torch.autograd import Variable

from keras.datasets import fashion_mnist
from keras.utils import np_utils

%matplotlib inline
import matplotlib.pylab as plt
import matplotlib as mpl

from torch.autograd import gradcheck

mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.figsize'] = (7, 7)
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['axes.labelsize'] = 12

# Get FashionMNIST (see 1b_FMNIST.ipynb for data exploration)
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Logistic regression needs 2D data
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# 0-1 normalization
x_train = x_train / 255.
x_test = x_test / 255.

# Convert to Torch Tensor. Just to avoid boilerplate code later
x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

# Use only first 1k examples. Just for notebook to run faster
x_train, y_train = x_train[0:1000], y_train[0:1000]
x_test, y_test = x_test[0:1000], y_test[0:1000]

In [ ]:
## Some helper functions

def get_model_weights(model):
    params = model.state_dict()
    p_order = [p[0] for p in sorted(model.named_parameters())]
    return np.concatenate([params[w].cpu().numpy().reshape(-1, ) for w in p_order])
    
def set_model_weights(model, w):
    params = model.state_dict()  
    p_order = [p[0] for p in sorted(model.named_parameters())]
    id = 0
    for p in p_order:
        shape = params[p].shape
        D = np.prod(shape)
        params[p].copy_(torch.from_numpy(w[id:id + D].reshape(shape)))
        id += D
        
def L(w, loss, model, batch_size=10, x_tr=x_train, y_tr=y_train):
    set_model_weights(model, w)
    cost = 0.
    n_examples, n_features = x_tr.size()
    num_batches = n_examples // batch_size
    for k in range(num_batches): 
        start, end = k * batch_size, (k + 1) * batch_size
        x = Variable(x_tr[start:end], requires_grad=False)
        y = Variable(y_tr[start:end], requires_grad=False)
        fx = model.forward(x)
        cost += loss.forward(fx, y)
    return cost.data.numpy()[0] / num_batches

# Training loop in PyTorch

In this section we walk through implementation of a traditional training loop  in PyTorch.

Necessary ingredients to train a neural network are:
    * model
    * loss
    * optimizer
    
<img width=400 src="https://github.com/gmum/nn2018/raw/master/lab/fig/4/smoothie.png">

Ref:

https://github.com/vinhkhuc/PyTorch-Mini-Tutorials/blob/master/3_neural_net.py

In [ ]:
def build_logreg(input_dim, output_dim, weight_module=torch.nn.Linear):
    model = torch.nn.Sequential()
    model.add_module("linear_2", weight_module(input_dim, output_dim, bias=False))
    return model

In [ ]:
def build_model(input_dim, output_dim, nonlinearity=torch.nn.functional.sigmoid, weight_module=torch.nn.Linear):
    class Nonlinearity(torch.nn.Module):
        def forward(self, input):
            return nonlinearity(input)
    
    model = torch.nn.Sequential()
    model.add_module("linear_1", weight_module(input_dim, 512, bias=False))
    model.add_module("nonlinearity_1", Nonlinearity())
    model.add_module("linear_2", weight_module(512, output_dim, bias=False))
    return model

In [ ]:
def sgd_step(model, loss, x_val, y_val, lr=0.1, momentum=0.9):
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    
    x = Variable(x_val, requires_grad=False)
    y = Variable(y_val, requires_grad=False)

    # Reset gradient
    optimizer.zero_grad()

    # Forward
    fx = model.forward(x)
    output = loss.forward(fx, y)

    # Backward
    output.backward(retain_graph=True)

    # Update parameters
    optimizer.step()

    return output.data[0]

In [ ]:
def predict(model, x_val):
    x = Variable(x_val, requires_grad=False)
    output = model.forward(x)
    return output.data.numpy().argmax(axis=1)

In [ ]:
def train(model, loss, step_fnc):
    torch.manual_seed(42)
    n_examples, n_features = x_train.size()
    n_classes = 10
    batch_size = 100
    history = []
    for i in tqdm.tqdm(range(100), total=100):
        cost = 0.
        num_batches = n_examples // batch_size
        for k in range(num_batches):
            start, end = k * batch_size, (k + 1) * batch_size
            cost += step_fnc(model, loss, x_train[start:end], y_train[start:end])
        predY = predict(model, x_test)
        print(("Epoch %d, cost = %f, acc = %.2f%%"
              % (i + 1, cost / num_batches, 100. * np.mean(predY == y_test.numpy()))))
        
        history.append(cost)
        
    return np.mean(predY == y_test.numpy()), history

## Run training

In [ ]:
model = build_model(n_features, n_classes)
loss = torch.nn.CrossEntropyLoss(size_average=True)
train(step_fnc=sgd_step, loss=loss, model=model)

# Estimating $\frac{\partial L (w) }{\partial w}$

In this section we will implement different gradient computation functions

## Finite Difference
<p>
<font size=4>
Assuming we know direction we want to optimize in, we can approximate derivative simply by calculating

$$\langle \frac{\partial L (w) }{\partial w}, \vec{v} \rangle \approx \frac{L(w) - L(w + v)}{|\vec{v}|}$$


, where $L(w)$ denotes esimation of loss (e.g. over whole training set, over $5\%$ etc.). 

</font>
</p>

<p>
<font size=4>
As we will see later PyTorch uses Finite Difference method in gradient checks of automatic diffentiation modules.
</font>
</p>

## Evolution Search

<p>
Intuitively, Evolution Search (ES) approximates gradient by computing multiple times finite difference. Importantly it works for non-differentiable costs as well.
</p>

<p>
<font size=4>
$$ \frac{\partial L (w) }{\partial w} \approx \frac{1}{N \sigma} \sum \vec{v} L(w + \vec{v})$$


, where $\epsilon$ is a normally distributed vector sampled from gauss of standard deviation $\sigma$, and $L(w)$ denotes esimation of loss (e.g. over whole training set, over $5\%$ etc.). 

</font>
</p>

<p>
<font size=4>
Note: while it is recommended to not use minibatching in ES, for simplicity we will.
</font>
</p>

### Variance reduction trick

Evolution Search works better (lower variance) if reward is standardized. A simple way to achieve it is to introduce a baseline:

<p>
<font size=4>
$$ \frac{\partial L (w) }{\partial w} \approx \frac{1}{N \sigma} \sum \vec{v} ( L(w + \vec{v}) - L(w) )$$

</font>
</p>

<p>
<font size=4>
Note that it has (as a random variable over random vectors) the same mean, but a different variance, compared to the previous expression.
</font>
</p>

## Implement Evolution Search

In [ ]:
def ES_grad(model, loss, x_tr, y_tr, sigma=0.001, N=100):
    """
    Estimate dL/dw using Evolution Search.
    
    Params
    ------
    model: torch.nn.Model
        Model to take optimization step on
    loss: torch.nn.Module
        Loss function to optimize
    x_tr: torch.Tensor, (n_examples, n_features)
        Batch to compute gradient over
    y_tr: torch.Tensor, (n_examples, )
        Batch to compute gradient over
    sigma: float
    
    N: int
        
    Returns
    -------
    g: np.vector, size: (D, )
        Gradient of loss with respect to model's weights
    """
    grad = 0
    init_w = get_model_weights(model)
    rng = np.random.RandomState()
    base_loss = L(w=init_w, model=model, loss=loss, x_tr=x_tr, y_tr=y_tr)
    for _ in tqdm.tqdm(range(N), total=N):
        v = ?? # Hint: use rng
        reward = ?? # Hint: use L
        grad += ?? # Hint: see equation
    return ?? # Hint: normalize!

In [ ]:
# If code is slow, you can use build_logreg !
# model = build_logreg(784, 10) 
model = build_model(784, 10)
loss = torch.nn.CrossEntropyLoss(size_average=True)
init_w = get_model_weights(model)

In [ ]:
down = 0
for _ in range(10):
    loss_before = L(w=init_w, model=model, loss=loss)
    set_model_weights(model=model, w=init_w)
    v = ES_grad(model, loss, x_tr=x_train, y_tr=y_train, sigma=0.002, N=50)
    loss_after = L(w=init_w - 0.1*v, model=model, loss=loss)
    if loss_before > loss_after:
        down += 1
        print("Went down!")
print("{}/{}".format(down, 10))
# This code is stochastic, so of course sometimes this assert can fail. 
assert down > 6

## Implement own module

PyTorch enable defining own Modules. Here we will implement Linear and ReQu modules ourselves, to understand better how backpropagation works. 

Below you can find an example Module implementing the classial ReLU nonlinearity.

Ref: https://github.com/jcjohnson/pytorch-examples

In [ ]:
class MyReLU(torch.autograd.Function):
  """
  We can implement our own custom autograd Functions by subclassing
  torch.autograd.Function and implementing the forward and backward passes
  which operate on Tensors.
  """
  def forward(self, input):
    """
    In the forward pass we receive a Tensor containing the input and return a
    Tensor containing the output. You can cache arbitrary Tensors for use in the
    backward pass using the save_for_backward method.
    """
    self.save_for_backward(input)
    return input.clamp(min=0)

  def backward(self, grad_output):
    """
    In the backward pass we receive a Tensor containing the gradient of the loss
    with respect to the output, and we need to compute the gradient of the loss
    with respect to the input.
    """
    input, = self.saved_tensors
    grad_input = grad_output.clone()
    grad_input[input < 0] = 0
    return grad_input

In [ ]:
# Gradient check passes!
input = (Variable(torch.randn(20,20).double(), requires_grad=True),)
assert(gradcheck(MyReLU(), input, eps=1e-6, atol=1e-4))

In [ ]:
model = build_model(n_features, n_classes, nonlinearity=MyReLU())
loss = torch.nn.CrossEntropyLoss(size_average=True)
_, H_relu = train(step_fnc=sgd_step, loss=loss, model=model)

### Implement “ReQU” unit

<p>
<font size=4>
Here, we’ll implement a made-up activation function that we’ll call the Rectified Quadratic Unit
(ReQU). Like the sigmoid and ReLU and several others, it is applied element-wise to all its
inputs:
</font>
</p>

<p>
<font size=4>
$$z_i = I[x_i > 0] x_i ^ 2$$
</font>
</p>

(Note, exercise is taken from  https://www.cs.ox.ac.uk/people/nando.defreitas/machinelearning/practicals/practical4.pdf)

In [ ]:
# Hint: use ReLU implementation as a template

class ReQU(torch.autograd.Function):
  def forward(self, input):
    self.save_for_backward(input)
    ??

  def backward(self, grad_output):
    """
    In the backward pass we receive a Tensor containing the gradient of the loss
    with respect to the output, and we need to compute the gradient of the loss
    with respect to the input.
    """
    input, = self.saved_tensors
    ??

In [ ]:
## GradCheck
input = (Variable(torch.randn(20,20).double(), requires_grad=True),)
assert(gradcheck(ReQU(), input, eps=1e-6, atol=1e-4))

In [ ]:
model = build_model(n_features, n_classes, nonlinearity=ReQU())
loss = torch.nn.CrossEntropyLoss(size_average=True)
_, H_requ = train(step_fnc=sgd_step, loss=loss, model=model)

In [ ]:
### Comparing ReLU and ReQU
plt.plot(H_relu, label="ReLU")
plt.plot(H_requ, label="ReQU")
plt.ylim([0, 10])
plt.legend()

### Implement W*x + b Function

(Code from PyTorch official tutorial)

#### Crash course on matrix derivatives

If A and B are matrices with elements being functions, then by (dA/dB)_{v1, v2} we understand dA[v1]/dB[v2], where v1 and v2 are two tuples of indices. Assuming this definition, chain rule applies to matrix operations, similarly as it would to normal functions.

Example useful for the exercise: A = WX + b (i.e. $A_i = \sum w_ik x_k + b_i$), then $\frac{\partial A}{\partial W}$ is simply $W$.

Hint: in the exercise you will have to consider dimension with respect to examples, which is ignored in this example (i.e. we assume here A is 1 dimensional, and X is 1 dimensional as well).

Refs:

* Matrix Cookbook: http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/3274/pdf/imm3274.pdf

In [ ]:
class Linear(torch.autograd.Function):
    def forward(self, input, weight, bias=None):
        # Hint: implement forward and check it manually, before implementing backward
        self.save_for_backward(input, weight, bias)
        output = ??
        if bias is not None:
            output += ??
        return output

    # This function has only a single output, so it gets only one gradient
    def backward(self, grad_output):
        input, weight, bias = self.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # Hint1: grad should have same shape as variable it is taken wrt to
        # Example: grad_bias should be 1 dimensional, because bias is a vectors
        
        # Hint2: you can use print, or even pdb inside backward pass
        # for example you can use print(grad_output.shape) 
        # Why is grad_output (20, 15)?
        
        # YOU DON'T HAVE TO IMPLEMENT GRAD_INPUT :)

        grad_weight = ??
        grad_bias = ?? 

        # dL/dinput_i !, where i=0 is input, i=1 is weight, i=2 is bias
        return None, grad_weight, grad_bias

In [ ]:
# Hint: if implementing bias gradient is difficult, consider removing from input triple last item
# this will test only gradient wrt to input and weight
input = (Variable(torch.randn(20,20).double(), requires_grad=False),  # Input
         Variable(torch.randn(15,20).double(), requires_grad=True), # Weight
         Variable(torch.randn(15,).double(), requires_grad=True)) # Bias
test = gradcheck(Linear(), input, eps=1e-6, atol=1e-4)
print(test)

# Tests

In [ ]:
result = {}

In [ ]:
input = (Variable(torch.randn(20,20).double(), requires_grad=True),)
result['requ'] = 0.5*int(gradcheck(ReQU(), input, eps=1e-6, atol=1e-4))

In [ ]:
input = (Variable(torch.randn(20,20).double(), requires_grad=False), 
         Variable(torch.randn(15,20).double(), requires_grad=True),
         Variable(torch.randn(15,).double(), requires_grad=True))
result['linear'] = 0.5*int(gradcheck(Linear(), input, eps=1e-6, atol=1e-4))

In [ ]:
# If code is slow, you can use build_logreg !
model = build_logreg(784, 10) 
loss = torch.nn.CrossEntropyLoss(size_average=True)
init_w = get_model_weights(model)
down = 0
for _ in range(10):
    loss_before = L(w=init_w, model=model, loss=loss)
    set_model_weights(model=model, w=init_w)
    v = ES_grad(model, loss, x_tr=x_train, y_tr=y_train, sigma=0.002, N=50)
    loss_after = L(w=init_w - 0.1*v, model=model, loss=loss)
    if loss_before > loss_after:
        down += 1
# This code is stochastic, so of course sometimes this assert can fail. 
result['ES'] = int(down > 6)

In [ ]:
result

In [ ]:
json.dump(result, open("4a_computing_gradient.json", "w"))